In [ ]:
import sqlite3
import telebot

# З'єднання з базою даних
conn = sqlite3.connect('messages.db')
cursor = conn.cursor()

# Створення таблиці повідомлень
cursor.execute('''
    CREATE TABLE IF NOT EXISTS messages
    (id INTEGER PRIMARY KEY, chat_id INTEGER, message TEXT)
''')

# Функція для зберігання повідомлень в базі даних
def save_message(message):
    chat_id = message.chat.id
    message_text = message.text
    cursor.execute('INSERT INTO messages(chat_id, message) VALUES(?, ?)', (chat_id, message_text))
    conn.commit()

# Створення бота
bot = telebot.TeleBot('5608244416:AAGO9fj5If9Focw1VttM2vIP7yDh7XYSRhs')

# Обробник повідомлень
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    save_message(message)

# Запуск бота
bot.polling()

In [ ]:
"""помилка зазвичай виникає тоді, коли ви використовуєте одне з'єднання з базою даних з різних потоків. Щоб уникнути
цієї помилки, вам потрібно зберігати окреме з'єднання з базою даних для кожного потоку.
Одним з способів цього досягнути є використання менеджера контексту with. Ось як можна модифікувати код для
використання менеджера контексту:"""

import sqlite3
import telebot
from threading import Thread

# Створення бота
bot = telebot.TeleBot('5608244416:AAGO9fj5If9Focw1VttM2vIP7yDh7XYSRhs')

# Функція для зберігання повідомлень в базі даних
def save_message(message):
    # З'єднання з базою даних
    with sqlite3.connect('messages.db') as conn:
        cursor = conn.cursor()

        # Створення таблиці повідомлень
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS messages
            (id INTEGER PRIMARY KEY, chat_id INTEGER, message TEXT)
        ''')

        # Збереження повідомлення в базі даних
        chat_id = message.chat.id
        message_text = message.text
        cursor.execute('INSERT INTO messages(chat_id, message) VALUES(?, ?)', (chat_id, message_text))
        conn.commit()

# Обробник повідомлень
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    thread = Thread(target=save_message, args=(message,))
    thread.start()

# Запуск бота
bot.polling()

"""У цьому варіанті коду save_message зберігає повідомлення в базі даних з використанням окремого з'єднання з базою
даних для кожного потоку. Код handle_message створює новий потік для збереження повідомлення в базі даних, щоб не
блокувати основний потік виконання бота."""

In [ ]:
Для збереження фото, відео та аудіо повідомлень від користувачів бота можна використовувати метод getFile() та бібліотеку urllib для отримання фото, відео та аудіо файлів з Telegram серверів. Далі ці файли можна зберегти на локальному комп'ютері, наприклад, в папці "media".

Нижче наведений код для отримання фото, аудіо та відео повідомлень і збереження їх на локальному комп'ютері:

python
Copy code
import urllib
import os
import telebot

TOKEN = "YOUR_TOKEN"
bot = telebot.TeleBot(TOKEN)

@bot.message_handler(content_types=['photo', 'audio', 'video'])
def save_files(message):
    # отримуємо інформацію про файл
    file_info = bot.get_file(message.document.file_id)
    file_path = file_info.file_path
    file_url = f"https://api.telegram.org/file/bot{TOKEN}/{file_path}"
    file_ext = os.path.splitext(file_path)[-1]
    
    # створюємо папку "media" якщо її не існує
    if not os.path.exists("media"):
        os.makedirs("media")
    
    # зберігаємо файл в папці "media" з назвою message_id і розширенням відповідного файлу
    filename = f"{message.message_id}{file_ext}"
    filepath = os.path.join("media", filename)
    urllib.request.urlretrieve(file_url, filepath)

    # відправляємо відповідь
    bot.reply_to(message, "Файл збережено")

bot.polling()